# CRAWLING

개발자 도구 탭 
- Elements - HTML 구조
- Console - Javascript 디버깅
- Sources - 웹페이지를 구성하는 src
- Performance - 웹페이지 성능 체크
- Network - 웹페이지에서 요청한 파일
- Memory - 웹페이지 메모리 사용률
- Application - 브라우저 스토리지정보 (Storage, Session, Cookie)

STATUS CODE
- 200 정상 작동
- 301 리다이렉트
- 404 사용자 오류
- 502 서버 오류

cf. urlencoding 

## 동적 웹 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {'User-Agent' :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

In [3]:
response = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=101&sid2=259", 
                        headers=headers 
                        # params={
                        #     'mode':'LS2D'
                        #     'mid' : 'shm',
                        #     'sid1' : 101,
                        #     'sid2' :259
                        # }
                        )
# requests.post()
# requests.put()
# requests.delete()  요청에 따라 다양한 메서드 사용 가능

✔ ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
headers 정보가 없으면 봇으로 인식하고 뜨는 에러 -> user-agent 정보로 headers 추가하기

In [4]:
print(response.status_code)


200


In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE HTML>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta contents="always" name="referrer"/>
<meta content="600" http-equiv="refresh">
<meta content="width=1106" name="viewport">
<meta content="경제 금융 : 네이버뉴스" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://news.naver.com/main/list.naver?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=259" property="og:url"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" property="og:image">
<meta content="각 언론사별 금융 기사 제공" property="og:description"/>
<meta content="네이버" property="og:article:author">
<meta content="summary" name="twitter:card"/>
<meta content="경제 금융 : 네이버뉴스" name="twitter:title"/>
<meta content="네이버 뉴스" name="twitter:site"/>
<meta content="네이버 뉴스" name="twitter:creator"/>
<meta content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_800x420_20221201.png" name="tw

- div 공백으로 나뉘어져 있으면 각각 다른 클래스
- ex) list_body newsflash_body -> list_body.newsflash_body 로 변경 필수

In [6]:
# newsDivTag = soup.select_one('div.list_body')
newsDivTag = soup.select_one('div.list_body.newsflash_body') # list_body newsflash_body -> list_body.newsflash_body 로 변경 필수

In [7]:
newsDivTag

<div class="list_body newsflash_body">
<ul class="type06_headline">
<li>
<dl>
<dt class="photo">
<a href="https://n.news.naver.com/mnews/article/421/0006926037?sid=101">
<img alt="CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/421/2023/07/13/6926037.jpg?type=nf106_72" width="106"/>
</a>
</dt>
<dt>
<a href="https://n.news.naver.com/mnews/article/421/0006926037?sid=101">
								 CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]
								</a>
</dt>
<dd>
<span class="lede">◇ 'CPI 시장 예상치 하회'에 뉴욕증시는 '활짝'했지만 비트코인은 하락세 가상자산(암호화폐) 대장주 비트코인이 '6월 미 소 …</span>
<span class="writing">뉴스1</span>
<span class="date is_new">3분전</span>
</dd>
</dl>
</li>
<li>
<dl>
<dt class="photo">
<a href="https://n.news.naver.com/mnews/article/001/0014064306?sid=101">
<img alt='한은 "논의 이르다"지만…4연속 동결에 금리 인하 기대↑' height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.new

In [8]:
liTags = newsDivTag.select('li')
liTags

[<li>
 <dl>
 <dt class="photo">
 <a href="https://n.news.naver.com/mnews/article/421/0006926037?sid=101">
 <img alt="CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/421/2023/07/13/6926037.jpg?type=nf106_72" width="106"/>
 </a>
 </dt>
 <dt>
 <a href="https://n.news.naver.com/mnews/article/421/0006926037?sid=101">
 								 CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]
 								</a>
 </dt>
 <dd>
 <span class="lede">◇ 'CPI 시장 예상치 하회'에 뉴욕증시는 '활짝'했지만 비트코인은 하락세 가상자산(암호화폐) 대장주 비트코인이 '6월 미 소 …</span>
 <span class="writing">뉴스1</span>
 <span class="date is_new">3분전</span>
 </dd>
 </dl>
 </li>,
 <li>
 <dl>
 <dt class="photo">
 <a href="https://n.news.naver.com/mnews/article/001/0014064306?sid=101">
 <img alt='한은 "논의 이르다"지만…4연속 동결에 금리 인하 기대↑' height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src

In [9]:
len(liTags)

20

In [10]:
liTags[0].select('dt')

[<dt class="photo">
 <a href="https://n.news.naver.com/mnews/article/421/0006926037?sid=101">
 <img alt="CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]" height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/421/2023/07/13/6926037.jpg?type=nf106_72" width="106"/>
 </a>
 </dt>,
 <dt>
 <a href="https://n.news.naver.com/mnews/article/421/0006926037?sid=101">
 								 CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]
 								</a>
 </dt>]

In [11]:
liTags[0].select('dt')[-1].text

'\n\n\t\t\t\t\t\t\t\t CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]\n\t\t\t\t\t\t\t\t\n'

In [12]:
liTags[0].select('dt')[-1].text.strip()

'CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]'

In [13]:
titles = []
for liTag in liTags:
    dtTag = liTag.select('dt')[-1]
    title = dtTag.text.strip()
    href = dtTag.select_one('a').attrs['href']
    # title = liTag.select('dt')[-1].text.strip() # ('dt')[1]로 설정할 경우 사진이 없는 기사도 있기 때문에 IndexError: list index out of range 에러남 -> ('dt')[-1] 로 변경
    # href = liTag.select('dt')[-1].select_one('a').attrs['href']
    print(href)
    titles.append(title)
titles

https://n.news.naver.com/mnews/article/421/0006926037?sid=101
https://n.news.naver.com/mnews/article/001/0014064306?sid=101
https://n.news.naver.com/mnews/article/003/0011971658?sid=101
https://n.news.naver.com/mnews/article/421/0006926028?sid=101
https://n.news.naver.com/mnews/article/119/0002730319?sid=101
https://n.news.naver.com/mnews/article/421/0006926017?sid=101
https://n.news.naver.com/mnews/article/014/0005041485?sid=101
https://n.news.naver.com/mnews/article/138/0002152249?sid=101
https://n.news.naver.com/mnews/article/014/0005041482?sid=101
https://n.news.naver.com/mnews/article/018/0005529282?sid=101
https://n.news.naver.com/mnews/article/662/0000024116?sid=101
https://n.news.naver.com/mnews/article/018/0005529274?sid=101
https://n.news.naver.com/mnews/article/648/0000017872?sid=101
https://n.news.naver.com/mnews/article/014/0005041476?sid=101
https://n.news.naver.com/mnews/article/014/0005041475?sid=101
https://n.news.naver.com/mnews/article/138/0002152247?sid=101
https://

['CPI 예상치 하회에도 하락세로 돌아선 비트코인 [코인브리핑]',
 '한은 "논의 이르다"지만…4연속 동결에 금리 인하 기대↑',
 'EY한영 "ESG 공시 \'잘 대비하고 있다\' 응답 16%에 그쳐"',
 "토스뱅크, '투자소식' 구독자 출시 한 달 반만에 구독자 40만명 달성",
 '이창용 한은 총재 "금통위원 전원 최종금리 3.75% 가능성 열어둬"',
 '신한은행, 직업계 고등학생 위한 신한 커리어온 3기 발대식 개최',
 '이복현 "상생 금융권 전반에 확산되길"..취약계층 지원 의지 강조',
 '‘토스뱅크 투자소식’, 구독자 40만명 달성',
 '이복현 금감원장 "글로벌 통화긴축 당분간 지속.. 상생금융 활성화해야"',
 '[인사]산업은행',
 '“경기 불확실성에 보험산업 둔화”…보험업계 희비 엇갈려',
 '코로나 끝나니 밖으로…가장 많이 찾은 카드 혜택은',
 "기준금리 4차례 연속 동결…인상 불씨는 '여전'",
 '\'정책 엇박자\' 일축한 이창용 "가계부채 우려, GDP 대비 80% 수준으로 내려야"',
 '금감원, 금융IT 비상대책 가이드라인 만든다',
 '중기부, 14일부터 햇살론 1000억원 규모 특례운용',
 '[일문일답] 이창용 "새마을금고 사태, 업권 아닌 개별기관 문제"',
 '한국콜마, 화장료 조성물 특허권 취득',
 '미원에스씨, 2분기 영업익 71.8억…78.13%↓',
 '금리 4번째 동결… 2주 뒤 한미 격차 2.00%P 벌어지나']

---

select 와 select_one 구분하기
- select : list 로 추출
- select_one : 일치하는 하나의 요소만 추출

In [14]:
dtTag.select('a') 

[<a href="https://n.news.naver.com/mnews/article/005/0001623258?sid=101">
 								 금리 4번째 동결… 2주 뒤 한미 격차 2.00%P 벌어지나
 								</a>]

In [15]:
dtTag.select('a')[0].attrs['href']

'https://n.news.naver.com/mnews/article/005/0001623258?sid=101'

In [16]:
dtTag.select_one('a').attrs['href']

'https://n.news.naver.com/mnews/article/005/0001623258?sid=101'

ID 셀렉터 - id 속성은 unique해서 주로 select_one 사용. #으로 접근
CLASS 셀렉터 - 동일한 요소를 가지고 있는 경우에 CLASS 로 묶으면 편집이 용이함(.사용)

---

## 실습

> 실습 1
- HTML 문서 내에 ID가 mw-content-text인 태그내의 내용을 출력해주세요.
- https://ko.wikipedia.org/wiki/위키백과

In [17]:
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC' # url encoding되어 있음
response = requests.get(url)
response # 동작이 잘 되는지 확인

<Response [200]>

In [18]:
soup = BeautifulSoup(response.text, 'html.parser')
result = soup.select_one('#mw-content-text').text  # id가 mw-content-text인 text 추출. id 셀렉터를 사용할 때는 select_one 활용하기

In [19]:
result

'\n\n  한국어 위키백과에 대해서는 한국어 위키백과 문서를, 다른 뜻에 대해서는 위키백과 (동음이의) 문서를 참고하십시오.\n위키백과위키백과의 로고각기 다른 언어의 위키백과를 표시하고 있는 위키백과 포털의 스크린샷웹사이트wikipedia.org 표어자유 백과사전영리여부비영리, 기부금을 통해 운영사이트 종류인터넷 백과사전, 위키회원 가입선택사용 언어317개 언어 (한국어 포함)사용자활동 사용자 >287,861명[주석 1]등록 사용자 >108,660,725명작성된 언어LAMP 플랫폼[1]소유자위키미디어 재단제작자지미 웨일스, 래리 생어시작일2001년 1월 15일(22년 전)(2001-01-15)현재 상태운영 중OCLC\xa0번호52075003 \n위키백과(-百科, IPA:\xa0[ɥikçibɛ̝k̚k͈wa̠], [ykçibɛ̝k̚k͈wa̠]\xa0( 듣기)) 또는 위키피디아(영어: Wikipedia, IPA:\xa0[ˌwɪkɪˈpiːdɪə]\xa0( 듣기))는 누구나 자유롭게 쓸 수 있는 다언어판 인터넷 백과사전이다.[2] 개방된 협업을 통해 위키 기반 편집 시스템을 사용하여 자발적인 위키백과 사용자 공동체가 작성하고 관리하고 있다. 위키백과는 대표적인 집단 지성의 사례로 평가받고 있으며[3] 사상 최대의, 가장 많이 읽힌 참고서이기도 하다.[4] 시밀러웹과 (과거) 알렉사 인터넷에서 가장 인기있는 웹사이트 10곳 중 하나로 꾸준히 순위를 올리고 있다. 2022년 기준으로 위키백과는 세계에서 5번째로 인기있는 사이트로 순위를 올렸다.[5] 위키백과는 자유 저작물을 보유하고 상업적인 광고가 없으며 주로 기부금을 통해 지원을 받는 비영리 단체인 위키미디어 재단이 소유하고 지원하고 있다.[6][7][8][9]\n2001년 1월 15일 지미 웨일스와 래리 생어가 창립하였다.[10] 생어는 위키(wiki)와 백과사전(encyclopedia)이라는 두 이름을 섞어서 \'위키피디아\'라는 이름을 만들어냈다.[11][12] 웨일스는 프리드리히 하이에크와 오스트리아 경제학파와

> 실습 2
- HTML 문서 내에 class가 list_item인 태그내의 내용을 출력해주세요
- https://www.saramin.co.kr/zf_user/jobs/public/list

In [20]:
url = "https://www.saramin.co.kr/zf_user/jobs/public/list"
response = requests.get(url, headers=headers)
response

<Response [200]>

In [21]:
print(response.text)

<!DOCTYPE html><html lang="ko">
<head>
        <title>실시간 공고 - 사람인</title>    <meta http-equiv="X-UA-Compatible" content="IE=Edge" >
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" >
<meta http-equiv="Content-Language" content="ko-KR" >
<meta name="naver-site-verification" content="86455485e27cab6986d130e4c3b90c5b516820d1" >
<meta name="Description" content="신입·인턴을 위한 실시간 공고 - 최신 업데이트 채용 정보, 합격률 높은 공고 추천, 관심 키워드별 공고 모두 사람인에서 확인해보세요." >
<meta name="writer" content="사람인" >
<meta name="keywords" content="신입, 인턴, 채용공고, 실시간 공고, 실시간 채용정보, 맞춤 채용정보, 신입모집, 인턴모집, 인턴십, 청년인턴, 스타트업, 인턴채용, 신입채용, 사람인, 대학생인턴, 대학생취업, 중고신입" >
<meta name="naver" content="nosublinks" >    <meta property="og:title" content="실시간 공고 - 사람인" >
<meta property="og:site_name" content="사람인" >
<meta property="og:url" content="https://www.saramin.co.kr/zf_user/jobs/public/list" >
<meta property="og:type" content="website" >
<meta property="og:description" content="신입·인턴을 위한 실시간 공고 - 최신 업데이트 채용 정보, 합격률 높은 공고 추천, 관심

In [22]:
soup = BeautifulSoup(response.text, 'html.parser')
items = soup.select('.list_item')  # class 이름이 list_item
items[0]

<div class="list_item" id="rec-46136267">
<div class="box_item">
<div class="col company_nm">
<a class="str_tit" href="/zf_user/company-info/view-inner-recruit?csn=SWxVaDVyUmRXSDVlcHZDWk5sbzdJQT09" target="_blank">
                        네이버클라우드(주)                    </a>
<button aria-pressed="false" class="interested_corp" csn="SWxVaDVyUmRXSDVlcHZDWk5sbzdJQT09" del_fl="n" first_nudge="off" onclick="try{Saramin.btnJob('favor', this, '', 'list');}catch(e){}" title="관심기업 등록" type="button"><span>관심기업 등록</span></button> <span class="main_corp" title="네이버그룹">네이버그룹</span>
<span class="info_stock" title="대기업">대기업</span>
</div>
<div class="col notification_info">
<div class="job_tit">
<a class="str_tit new" href="/zf_user/jobs/relay/view?view_type=public-recruit&amp;rec_idx=46136267" id="rec_link_46136267" onclick="s_trackApply(this, 'public-recruit', 'generic');" onmousedown="" target="_blank" title="[Hyperscale AI] AI 사업 개발 인턴십 모집(체험형)"><span>[Hyperscale AI] AI 사업 개발 인턴십 모집(체험형)</span></a> 

In [23]:
for item in items:
    print(item.text)





                        네이버클라우드(주)                    
관심기업 등록 네이버그룹
대기업



[Hyperscale AI] AI 사업 개발 인턴십 모집(체험형) 스크랩 


AI(인공지능)문서작성사무보조PowerPoint사업기획 





경기 성남시 분당구


신입 · 인턴직


대학교(4년)↑



                                면접 후 협의                            





자소서작성
 
~07.26(수)
3시간 전 등록









                        (주)에코마케팅                    
관심기업 등록 에코마케팅그룹



직무별 상시채용(영상) 스크랩 


영상제작영상편집촬영보조콘텐츠기획모션그래픽 외                    





서울 강남구


신입 · 경력 · 정규직


학력무관



                                면접 후 협의                            





자소서작성
 
~07.31(월)
3시간 전 등록









                        (주)에코마케팅                    
관심기업 등록 에코마케팅그룹



글로벌 콘텐츠 마케팅(일본어) 체험형 인턴 채용 스크랩 


일본어마케팅기획글로벌마케팅콘텐츠마케팅트렌드분석 외                    





서울 강남구


신입 · 인턴직


학력무관



                                면접 후 협의                            





자소서작성
 
~07.31(월)
4시간 전 등록









                        (주)드림씨아이에스                    
관심기업 등록 드림씨아이에스그룹
외국계



[서울본사] Data Management CDPL/CDA 신입

- 속성 셀렉터
    - 태그 내의 속성을 활용
    - 셀렉터[속성=“값“], 정확히 일치
    - 셀렉터[속성~=“값”], 해당 단어를 포함
    - 셀렉터[속성^=“값“], 해당 값으로 시작
    - 셀렉터[속성$=“값“], 해당 값으로 끝
    - 셀렉터[속성*=“값”], 해당 값을 포함
- 자식 셀렉터 : '>' 사용  [class^='type06']>li
- 후손 셀렉터 : 공백 사용  [class^='type06'] li

---

## 실습

> 실습 1
- 사이트 내 공지사항을 하나씩 출력해주세요.
- https://didimteo.startup-plus.kr/default.do


In [24]:
import requests
url = "https://didimteo.startup-plus.kr/cms_for_bcb/process/notice/list.do?show_no=2326&check_no=2312&c_relation=35&c_relation2=23"
response = requests.get(url, headers=headers)
response

<Response [200]>

In [25]:
soup = BeautifulSoup(response.text, 'html.parser')
trTags = soup.select('tbody>tr')  # thead>tr은 제외할 수 있도록

In [26]:
for trTag in trTags:
    print(trTag.select_one('.list-title').text.strip())

[창업교육·멘토링] 제2회 정부지원 창업 도전 : 사업계획서 교육생 모집 안내
[창업교육·멘토링] 제1회 창업새내기 : 직무전환/재창업/홍보·마케팅 교육생 모집 안내
[공사] 5/20~6/20 화장실 리모델링 공사 안내
[서울창업디딤터] 2022년 서울창업디딤터 입주기업 홍보 책자
[서울창업디딤터] 광운대학교 산학협력단 직원 채용 공고
[BI 추가모집] 2022년 서울창업디딤터 BI 입주기업 모집
2022년 서울창업디딤터 데모데이(DEMO DAY)
[서울창업디딤터] 2022년 카페 운영사업자 공모 안내 (~11/29, 화)
2022년 서울창업디딤터 에듀테크 입주기업 추가 모집
[서울창업디딤터] 2021년 PRE-BI 입주기업 추가 모집 공고 (~8/13,14시)


> 실습 2
- 실습 1의 결과에서 제목과 날짜를 분리해 딕셔너리 형태로 저장해주세요
- 결과 : [{'제목': '[채용] 광운대학교 산학협력단(서울창업디딤터) 직원 채용 공고(~6/17)', '날짜': '2022.06.10'},
{'제목': '[서울창업디딤터] 대강의실 및 공동작업실 이용 신청 방식 변경', '날짜': '2022.06.04’}, ... ]

In [27]:
for trTag in trTags:
    print(trTag.select_one('.list-title').text.strip())
    print(trTag.select_one('.list-right').text.strip())

[창업교육·멘토링] 제2회 정부지원 창업 도전 : 사업계획서 교육생 모집 안내
2023-06-27
[창업교육·멘토링] 제1회 창업새내기 : 직무전환/재창업/홍보·마케팅 교육생 모집 안내
2023-06-08
[공사] 5/20~6/20 화장실 리모델링 공사 안내
2023-05-19
[서울창업디딤터] 2022년 서울창업디딤터 입주기업 홍보 책자
2022-12-23
[서울창업디딤터] 광운대학교 산학협력단 직원 채용 공고
2022-12-07
[BI 추가모집] 2022년 서울창업디딤터 BI 입주기업 모집
2022-11-28
2022년 서울창업디딤터 데모데이(DEMO DAY)
2022-11-18
[서울창업디딤터] 2022년 카페 운영사업자 공모 안내 (~11/29, 화)
2022-11-16
2022년 서울창업디딤터 에듀테크 입주기업 추가 모집
2022-08-31
[서울창업디딤터] 2021년 PRE-BI 입주기업 추가 모집 공고 (~8/13,14시)
2021-08-03


In [28]:
result = []
for trTag in trTags:
    temp = {
        '제목' : trTag.select_one('.list-title').text.strip(),
        '날짜' : trTag.select_one('.list-right').text.strip()
    }
    result.append(temp)
result

[{'제목': '[창업교육·멘토링] 제2회 정부지원 창업 도전 : 사업계획서 교육생 모집 안내', '날짜': '2023-06-27'},
 {'제목': '[창업교육·멘토링] 제1회 창업새내기 : 직무전환/재창업/홍보·마케팅 교육생 모집 안내',
  '날짜': '2023-06-08'},
 {'제목': '[공사] 5/20~6/20 화장실 리모델링 공사 안내', '날짜': '2023-05-19'},
 {'제목': '[서울창업디딤터] 2022년 서울창업디딤터 입주기업 홍보 책자', '날짜': '2022-12-23'},
 {'제목': '[서울창업디딤터] 광운대학교 산학협력단 직원 채용 공고', '날짜': '2022-12-07'},
 {'제목': '[BI 추가모집] 2022년 서울창업디딤터 BI 입주기업 모집', '날짜': '2022-11-28'},
 {'제목': '2022년 서울창업디딤터 데모데이(DEMO DAY)', '날짜': '2022-11-18'},
 {'제목': '[서울창업디딤터] 2022년 카페 운영사업자 공모 안내 (~11/29, 화)', '날짜': '2022-11-16'},
 {'제목': '2022년 서울창업디딤터 에듀테크 입주기업 추가 모집', '날짜': '2022-08-31'},
 {'제목': '[서울창업디딤터] 2021년 PRE-BI 입주기업 추가 모집 공고 (~8/13,14시)',
  '날짜': '2021-08-03'}]

> 실습 3
- 사이트에서 하이퍼링크에 baCategory1=basic이 포함된 태그를 하나만 선택해 출력해주세요.
- https://youth.seoul.go.kr/site/main/home

In [29]:
url = 'https://youth.seoul.go.kr/site/main/home'
response = requests.get(url, headers=headers)
response

<Response [200]>

In [30]:
soup = BeautifulSoup(response.text, 'html.parser')
items = soup.select("[href*='baCategory1=basic']")

In [31]:
for item in items:
    print(item.text.strip())

공지사항
공지사항
공지사항
2023년 서울 청년정책 콘테스트 피칭 라운드 진출자 선정 결과
2023년 청년인생설계학교 3기 참여자 추첨 결과 및 대면 인터뷰 대상자 공지
하절기 에너지절약 제로서울 온라인 챌린지
청년수당 참여자 2회차 자기활동기록서 작성 기간 안내(6/25~6/30)





> 실습 4
- 사이트에서 주요뉴스 내용과 일자별 뉴스들을 리스트로 정리해주세요
- https://ko.wikipedia.org/wiki/포털:요즘_화제
- 결과 : [‘주요 뉴스’, '8월 25일, 파키스탄에서 일어난 홍수로 1,000명 이상의 주민이 사망하고, 가축 70,000마리 이상이 죽었다.’, ..., 
'Current events of 2022년 8월 29일\xa0(2022-08-29) (월요일)', ‘대한민국의 0시 기준 누적 \u200b확진자 수가 23,026,960명으로
집계되었다. 전날 0시 대비 43,142명(국내 42,782, 해외유입 360)이 늘었다.’, ...]

In [32]:
url = "https://ko.wikipedia.org/wiki/%ED%8F%AC%ED%84%B8:%EC%9A%94%EC%A6%98_%ED%99%94%EC%A0%9C"
response = requests.get(url, headers=headers)
response

<Response [200]>

In [37]:
soup = BeautifulSoup(response.text, 'html.parser')
tableTags = soup.select("[class='vevent']") # soup.select('.vevent')

In [43]:
len(tableTags)

19

In [68]:
mainNews = ['주요 뉴스']
tableTags[0]   # 주요 뉴스 

liTags = tableTags[0].select('li')
for li in liTags:
    mainNews.append(li.text.strip())

mainNews


['주요 뉴스',
 '7월 11일, 리투아니아 빌뉴스에서 2일간 나토 정상 회의가 열렸다.',
 '7월 1일, 서해선 소사역~대곡역 구간이 개통되었다.',
 '6월 30일, 브라질의 선거법원이 자이르 보우소나루 전 대통령 (사진)을 2022년 대선 전 직권남용 건으로 2030년까지 공직에 출마하는 것을 금지하였다.',
 '6월 27일, 프랑스에서 발생한 나헬 메르주크 살해 사건으로 대규모 소요사태가 발생했다.',
 '6월 24일, 바그너 그룹의 예브게니 프리고진이 러시아 정부를 상대로 한 반란을 선포했다. 이후 알렉산드르 루카셴코 벨라루스 대통령의 중재로 반란은 철회되었다.']

In [70]:
resutl = [mainNews]
for tableTag in tableTags[1:-3]:  # 일자별 뉴스 반복문 처리
    result.append([tableTag.select_one('.summary').text.strip(),    # list extend와 list append 차이
                  tableTag.select_one('.description').text.strip()])
result

['주요 뉴스',
 '7월 11일, 리투아니아 빌뉴스에서 2일간 나토 정상 회의가 열렸다.',
 '7월 1일, 서해선 소사역~대곡역 구간이 개통되었다.',
 '6월 30일, 브라질의 선거법원이 자이르 보우소나루 전 대통령 (사진)을 2022년 대선 전 직권남용 건으로 2030년까지 공직에 출마하는 것을 금지하였다.',
 '6월 27일, 프랑스에서 발생한 나헬 메르주크 살해 사건으로 대규모 소요사태가 발생했다.',
 '6월 24일, 바그너 그룹의 예브게니 프리고진이 러시아 정부를 상대로 한 반란을 선포했다. 이후 알렉산드르 루카셴코 벨라루스 대통령의 중재로 반란은 철회되었다.',
 ['Current events of 2023년 7월 12일\xa0(2023-07-12) (수요일)', ''],
 ['Current events of 2023년 7월 11일\xa0(2023-07-11) (화요일)', ''],
 ['Current events of 2023년 7월 10일\xa0(2023-07-10) (월요일)', ''],
 ['Current events of 2023년 7월 9일\xa0(2023-07-09) (일요일)', ''],
 ['Current events of 2023년 7월 8일\xa0(2023-07-08) (토요일)', ''],
 ['Current events of 2023년 7월 7일\xa0(2023-07-07) (금요일)',
  "법과 범죄\n대한민국 영아 살인 사건: 감사원은 보건복지부 정기감사에서 2015년부터 2022년까지 8년 간 의료기관 출생 기록은 있지만 행정기관에 신고되지 않은 '유령 영ㆍ유아' 2,236명을 소재 파악했다."],
 ['Current events of 2023년 7월 6일\xa0(2023-07-06) (목요일)', ''],
 ['Current events of 2023년 7월 5일\xa0(2023-07-05) (수요일)', ''],
 ['Current events of 2023년 7월 4일\xa0(2023-07-04) (화요일)'